In [2]:
# Create connection
import mysql.connector

connection = mysql.connector.connect(
    user='root',
    password='password',
    host='localhost',
    database='db'
)

# Make sure connection established
if connection.is_connected():
    print("Connected")
else:
    print("Failed to connect")

Connected


## Queries

#### Roll-Up/Drill Down

Roll up car sales by car make and model.

In [3]:
cursor = connection.cursor()

In [4]:
query = 'SELECT c.company, c.model, SUM(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car GROUP BY c.company, c.model;'
cursor.execute(query)
cursor.fetchall()

[('ford', 'expedition', Decimal('4838250')),
 ('dodge', 'durango', Decimal('5878455')),
 ('cadillac', 'eldorado', Decimal('9725407')),
 ('toyota', 'celica', Decimal('4584448')),
 ('acura', 'tl', Decimal('6139019')),
 ('mitsubishi', 'diamante', Decimal('9308151')),
 ('toyota', 'corolla', Decimal('2878632')),
 ('mitsubishi', 'galant', Decimal('3255171')),
 ('chevrolet', 'malibu', Decimal('8885102')),
 ('ford', 'escort', Decimal('980483')),
 ('acura', 'rl', Decimal('8778638')),
 ('nissan', 'pathfinder', Decimal('8792523')),
 ('subaru', 'forester', Decimal('5738743')),
 ('hyundai', 'accent', Decimal('634518')),
 ('toyota', 'landcruiser', Decimal('2349022')),
 ('honda', 'accord', Decimal('6597822')),
 ('toyota', '4runner', Decimal('1695515')),
 ('infiniti', 'i30', Decimal('5717040')),
 ('audi', 'a4', Decimal('2009977')),
 ('volkswagen', 'jetta', Decimal('12287736')),
 ('dodge', 'viper', Decimal('7417262')),
 ('buick', 'regal', Decimal('4760614')),
 ('chrysler', 'lhs', Decimal('9097717')),
 

Roll up car sales by year and fiscal quarter.

In [21]:
query = 'CREATE VIEW salesbyfuelincome AS SELECT d.year, d.fiscal_quarter, SUM(cs.price) AS total_price FROM car_sales cs JOIN date d ON cs.date = d.id GROUP BY d.year, d.fiscal_quarter;'
cursor.execute(query)
cursor.fetchall()

[]

#### Slice

Select sales where the CPI value is greater than 120.

In [6]:
query = 'SELECT cs.*FROM car_sales cs JOIN date d ON cs.date = d.id JOIN cpi c ON d.id = c.date WHERE c.cpi_value > 120;'
cursor.execute(query)
cursor.fetchall()

[(496, 67884, 14821, 14412, 13952, 39501),
 (497, 67781, 14821, 14413, 13942, 19001),
 (498, 67817, 14821, 14414, 13933, 21001),
 (499, 67885, 14821, 14415, 13926, 12000),
 (500, 67733, 14821, 14416, 13928, 20500),
 (501, 67850, 14821, 14417, 13921, 14000),
 (502, 67652, 14821, 14418, 13922, 14100),
 (503, 67842, 14821, 14419, 13956, 12001),
 (504, 67886, 14821, 14420, 13924, 19000),
 (505, 67659, 14821, 14421, 13925, 34300),
 (506, 67689, 14821, 14422, 13957, 36000),
 (507, 67861, 14821, 14423, 13951, 26000),
 (508, 67887, 14821, 14424, 13951, 22000),
 (509, 67720, 14821, 14425, 13959, 41000),
 (510, 67888, 14821, 14426, 13956, 31000),
 (511, 67654, 14821, 14427, 13935, 25100),
 (512, 67889, 14821, 14428, 13928, 51200),
 (513, 67657, 14821, 14429, 13920, 14000),
 (514, 67739, 14821, 14430, 13926, 11000),
 (515, 67757, 14821, 14431, 13919, 26000),
 (516, 67654, 14821, 14432, 13920, 19000),
 (517, 67763, 14821, 14433, 13921, 31500),
 (518, 67731, 14821, 14434, 13955, 46000),
 (519, 6771

#### Dice

Select sales where the city miles per gallon of the car purchased is less than 19 and the income of the customer is less than 60000

In [7]:
cursor.fetchall()
query = '''
SELECT cs.*
FROM car_sales cs
JOIN car c ON cs.car = c.id
JOIN customer cu ON cs.customer = cu.id
WHERE c.city_mpg < 19 AND cu.annual_income < 60000;
'''
cursor.execute(query)
cursor.fetchall()

[(3, 67649, 14844, 13919, 13919, 26000),
 (13, 67659, 14844, 13929, 13929, 31000),
 (20, 67666, 14844, 13936, 13925, 25000),
 (27, 67672, 14844, 13943, 13936, 41000),
 (32, 67676, 14845, 13948, 13940, 61000),
 (41, 67685, 14845, 13957, 13944, 22000),
 (46, 67688, 14846, 13962, 13949, 17000),
 (51, 67692, 14846, 13967, 13933, 26000),
 (54, 67694, 14846, 13970, 13955, 21000),
 (71, 67709, 14847, 13987, 13929, 26000),
 (75, 67713, 14847, 13991, 13922, 16000),
 (82, 67719, 14848, 13998, 13929, 16001),
 (86, 67722, 14848, 14002, 13923, 49000),
 (96, 67730, 14849, 14012, 13951, 12000),
 (104, 67736, 14850, 14020, 13926, 20000),
 (116, 67727, 14851, 14032, 13929, 69000),
 (120, 67747, 14851, 14036, 13945, 39000),
 (123, 67749, 14851, 14039, 13949, 19000),
 (125, 67751, 14851, 14041, 13951, 71000),
 (126, 67703, 14851, 14042, 13959, 25000),
 (128, 67753, 14851, 14044, 13972, 22000),
 (136, 67758, 14852, 14052, 13952, 82000),
 (137, 67724, 14852, 14053, 13942, 24000),
 (156, 67670, 14853, 14072

Select sales where the engine fuel type is "regular unleaded" and customer annual income is > 82000

In [8]:
query = '''
SELECT cs.*
FROM car_sales cs
JOIN car c ON cs.car = c.id
JOIN customer cu ON cs.customer = cu.id
WHERE c.engine_fuel_type = 'regular unleaded' AND cu.annual_income > 82000;
'''
cursor.execute(query)
cursor.fetchall()

[(384, 67649, 14870, 14300, 13944, 26001),
 (752, 67649, 14891, 14668, 13944, 26000),
 (1499, 67649, 14920, 15415, 13930, 41100),
 (1866, 67649, 14938, 15782, 13923, 41000),
 (2205, 67649, 14955, 16121, 13945, 41001),
 (2388, 67649, 14961, 16304, 13925, 41000),
 (2776, 67649, 14977, 16692, 13952, 26000),
 (2962, 67649, 14985, 16878, 13924, 26000),
 (3253, 67649, 14999, 17169, 13929, 41000),
 (3645, 67649, 15013, 17561, 13935, 41000),
 (3678, 67649, 15013, 17594, 13971, 26000),
 (4430, 67649, 15035, 18346, 13923, 41001),
 (4472, 67649, 15035, 18388, 13935, 41000),
 (5038, 67649, 15045, 18954, 13957, 26000),
 (5699, 67649, 15070, 19615, 13921, 41000),
 (5915, 67649, 15075, 19831, 13977, 26550),
 (6855, 67649, 15095, 20771, 13943, 26001),
 (7014, 67649, 14830, 20930, 13928, 26000),
 (7225, 67649, 15102, 21141, 13928, 26001),
 (7650, 67649, 15110, 21566, 14049, 26000),
 (8062, 67649, 15121, 21978, 14002, 26000),
 (8135, 67649, 15123, 22051, 13919, 41000),
 (8348, 67649, 15135, 22264, 13941

#### Iceberg

Select the top 10% of car models based on their total sales. 

In [9]:
query = '''SELECT *
FROM (
  SELECT c.model, SUM(cs.price) AS TotalSales, @row_num := @row_num + 1 AS row_num
  FROM car_sales cs
  JOIN car c ON cs.car = c.id
  CROSS JOIN (SELECT @row_num := 0) AS rn
  GROUP BY c.model
  ORDER BY TotalSales DESC
) AS ranked_data
WHERE row_num <= (
  SELECT FLOOR(COUNT(DISTINCT model) * 0.1) FROM car
);'''

cursor.execute(query)
cursor.fetchall()

[('eldorado', Decimal('9725407'), 3.0),
 ('diamante', Decimal('9308151'), 6.0),
 ('malibu', Decimal('8885102'), 9.0),
 ('rl', Decimal('8778638'), 11.0),
 ('tl', Decimal('6139019'), 5.0),
 ('durango', Decimal('5878455'), 2.0),
 ('expedition', Decimal('4838250'), 1.0),
 ('celica', Decimal('4584448'), 4.0),
 ('galant', Decimal('3255171'), 8.0),
 ('corolla', Decimal('2878632'), 7.0),
 ('escort', Decimal('980483'), 10.0)]

#### Window

Select car company and number of sales. Rank the car companies in descending order by number of sales. 

In [10]:
query = '''SELECT c.company, COUNT(cs.id) AS NumberOfSales, 
RANK() OVER (ORDER BY COUNT(cs.id) DESC) AS sales_rank 
FROM car_sales cs 
JOIN car c ON cs.car = c.id 
GROUP BY c.company;'''
cursor.execute(query)
cursor.fetchall()

[('chevrolet', 1819, 1),
 ('ford', 1504, 2),
 ('volkswagen', 1333, 3),
 ('dodge', 1288, 4),
 ('mitsubishi', 1277, 5),
 ('oldsmobile', 1111, 6),
 ('toyota', 1110, 7),
 ('nissan', 886, 8),
 ('chrysler', 847, 9),
 ('lexus', 802, 10),
 ('pontiac', 796, 11),
 ('volvo', 789, 12),
 ('honda', 708, 13),
 ('acura', 689, 14),
 ('cadillac', 652, 15),
 ('plymouth', 617, 16),
 ('lincoln', 492, 17),
 ('audi', 468, 18),
 ('buick', 439, 19),
 ('subaru', 405, 20),
 ('hyundai', 264, 21),
 ('infiniti', 195, 22)]

This query provides a running total of sales prices ordered by date

In [11]:
query = '''SELECT d.month, d.day, d.year, cs.price, SUM(cs.price) 
OVER (ORDER BY cs.date) AS RunningTotal 
FROM car_sales cs 
JOIN date d ON cs.date = d.id
ORDER BY cs.date; ''' 

## Combining OLAP Operations

### Slice + Dice + Roll Up

In [12]:
# Within the past 3 months for Toyota models grouped by month and year
query = "SELECT d.month, d.year, c.company, c.model, SUM(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE (d.year = 2023 AND d.month >= 10 AND d.month <= 12 AND d.day <= 31) AND c.company = 'Toyota' GROUP BY d.year, d.month, c.model;"

# Within the past year for Toyota models grouped by month and year
query = "SELECT d.month, d.year, c.company, c.model, SUM(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE d.year = 2023 AND c.company = 'Toyota' GROUP BY d.year, d.month, c.model;"


### Slice + Dice

In [13]:
# Female
query = "SELECT cst.annual_income, cr.engine_fuel_type FROM customer cst JOIN car_sales cs ON cst.id = cs.customer JOIN car cr ON cs.car = cr.id WHERE cst.gender = 'Female';"

# Male
query = "SELECT cst.annual_income, cr.engine_fuel_type FROM customer cst JOIN car_sales cs ON cst.id = cs.customer JOIN car cr ON cs.car = cr.id WHERE cst.gender = 'Male';"

### Dice + Roll Up

In [14]:
# Get number of sales for a month for all Toyotas
query = "SELECT d.month, c.company, COUNT(cs.id), AVG(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE c.company = 'Toyota' GROUP BY d.month, c.company;"

### Drill Down + Slice

In [15]:
query = "SELECT d.month, d.day, c.company, COUNT(cs.id), AVG(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE c.company = 'Toyota' GROUP BY d.month, d.day, c.company;"